<a href="https://colab.research.google.com/github/cricketplayer/CollabNotebooks/blob/main/Mistral7llamaindexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.5/276.5 kB 5.6 MB/s eta 0:00:00


In [2]:
!pip install -q transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 205.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423483 sha256=00cf0f286ee5070308b54179c877bfdc93cb638d8eacf084da6a0916393e5c60
  Stored in directory: /tmp/pip-ephem-wheel-cache-dd6vxpvp/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


In [4]:
!pip install -q llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.1/868.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


In [20]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [21]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [22]:
!pip -q install sentence-transformers

In [23]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


In [24]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

In [25]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [26]:
query_engine = index.as_query_engine()
response = query_engine.query("Tell me about Forrester Consulting")

In [27]:
print(response)

 Forrester Consulting is an independent and objective research-based consulting firm that provides leaders with insights and guidance to help them succeed in their organizations. They offer a range of services including strategy, operations, technology, and data management. Their research is based on the best available resources and their opinions reflect judgment at the time but are subject to change. Forrester Consulting has several trademarks including Forrester®, Technographics®, Forrester Wave, RoleView, TechRadar, and Total Economic Impact.


In [29]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

What were the key challenges for deploying dynamics 365


Llama.generate: prefix-match hit


 The key challenges for deploying Dynamics 365 Sales in the interviewees' companies were excessive manual work that hindered the efficiency of sales-related activities and took time away from higher-value efforts.
what else


Llama.generate: prefix-match hit


 I apologize, but I am unable to provide an answer to your query without additional context or specific instructions. Could you please clarify what you are asking for?
give me a few key challenges that companies come across before deploying dynamics 365


Llama.generate: prefix-match hit


 Based on the provided context information, some of the key challenges that companies come across before deploying Dynamics 365 include excessive manual work that hinders the efficiency of sales-related activities and takes time away from higher-value efforts. Additionally, companies may struggle with common challenges such as relying on highly manual and disjointed processes involving various mixes of spreadsheets, emails, office productivity tools, and information posted to internal websites, or homegrown CRM systems that aren't sufficiently addressing current needs.
give me evidence based data on incremental operating profit from increased sales


Llama.generate: prefix-match hit


 Based on the provided context information, the incremental operating profit from increased sales can vary based on several factors such as the extent to which a company capitalizes on Dynamics 365 Sales' capabilities, prior state of sales processes and systems, number of sellers, prior state for number of opportunities addressed and close rates, headwinds or tailwinds from prevailing economic conditions, competitive landscape, current offerings, customer experience, average deal size, and the organization's operating margin.

To account for these risks, Forrester adjusted this benefit downward by 20%, yielding a three-year, risk-adjusted total PV (discounted at 10%) of about $51.0 million. However, without specific data on the company's sales processes and systems, number of sellers, prior state for number of opportunities addressed and close rates, headwinds or tailwinds from prevailing economic conditions, competitive landscape, current offerings, customer experience, average deal s

KeyboardInterrupt: ignored